ref. https://www.codewars.com/kata/52b7ed099cdc285c300001cd, (4 kyu) - 2019-12-06

Write a function called `sum_intervals()` that accepts an array of intervals, and returns the sum of all the interval lengths. Overlapping intervals should only be counted once.

**Intervals**:  

Intervals are represented by a pair of integers in the form of a tuple. The first value of the interval will always be less than the second value. Interval example: `(1, 5)` is an interval from 1 to 5. The length of this interval is 4.


**Overlapping Intervals**:  
List containing overlapping intervals:

```julia
[
  (1, 4),
  (7, 10),
  (3, 5)
]
```

The sum of the lengths of these intervals is 7. Since `(1, 4)` and `(3, 5)` overlap, we can treat the interval as `(1, 5)`, which has a length of 4.


**Examples**:  

```julia
sum_intervals([
   (1, 2),
   (6, 10),
   (11, 15)
]); # => 9

sum_intervals([
   (1, 4),
   (7, 10),
   (3, 5)
]); # => 7

sum_intervals([
   (1, 5),
   (10, 20),
   (1, 6),
   (16, 19),
   (5, 11)
]); # => 19
```


Ok, let's use a a `struct` with parametric type  

In [1]:
using Test
using BenchmarkTools

#### Defining parametric type interval

In [2]:
struct Interval{T <: Integer}
    lb::T  # lower bound
    ub::T  # upper bound

    Interval{T}(lb, ub) where {T <: Integer} = lb > ub ? new(ub, lb) : new(lb, ub)
end

In [3]:
IntervInt64 = Interval{Int64}

Interval{Int64}

In [4]:
# interv1(lb::Int64, ub::Int64) = Interval(2, 1)
interv1 = IntervInt64(3, 1)

Interval{Int64}(1, 3)

##### Compute length of an interval

In [5]:
function interv_length(interv::Interval{T})::T where T <: Integer 
    return interv.ub - interv.lb
end

interv_length (generic function with 1 method)

##### Instanciate a concrete type interval

In [6]:
interv1 = IntervInt64(3, 1)
@test interv_length(interv1) == 2

interv2 = IntervInt64(4, 7)
@test interv_length(interv2) == 3

interv3 = IntervInt64(1, 1)
@test interv_length(interv3) == 0

interv4 = IntervInt64(5, 1) # re-order as IntervInt64(1, 5)
@test interv_length(interv4) == 4

Test Passed

#### Compute if two intervals overlap

In [7]:
function interv_inter(interv1::Interval{T}, interv2::Interval{T})::Bool where T <: Integer
   lb1, ub1 = interv1.lb, interv1.ub
   lb2, ub2 = interv2.lb, interv2.ub

   (lb1 ≤ lb2 ≤ ub2 ≤ ub1) || 
     (lb2 ≤ lb1 ≤ ub1 ≤ ub2) ||
     (lb1 ≤ lb2 ≤ ub1 ≤ ub2) ||
     (lb2 ≤ lb1 ≤ ub2 ≤ ub1)
end

interv_inter (generic function with 1 method)

In [8]:
interv_1 = IntervInt64(1, 10)
interv_2 = IntervInt64(3, 7)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(3, 7)
interv_2 = IntervInt64(1, 8)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(1, 6)
interv_2 = IntervInt64(3, 9)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(1, 5)
interv_2 = IntervInt64(3, 4)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(1, 5)
interv_2 = IntervInt64(5, 6)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(6, 10)
interv_2 = IntervInt64(5, 8)
@test interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(1, 5)
interv_2 = IntervInt64(6, 8)
@test !interv_inter(interv_1, interv_2)

interv_1 = IntervInt64(1, 5)
interv_2 = IntervInt64(10, 20)
@test !interv_inter(interv_1, interv_2)

Test Passed

####  Adding two interval length

In [9]:
function add_interv_len(interv1::Interval{T}, interv2::Interval{T})::T where T <: Integer
   lb1, ub1 = interv1.lb, interv1.ub
   lb2, ub2 = interv2.lb, interv2.ub

   (lb1 ≤ lb2 ≤ ub2 ≤ ub1) && return ub1 - lb1 # interv1 encompasses interv2
   (lb2 ≤ lb1 ≤ ub1 ≤ ub2) && return ub2 - lb2 # interv2 encompasses interv1 - symmetry
     
   sum_inter = interv_length(interv1) + interv_length(interv2)
    
   (lb1 ≤ lb2 ≤ ub1 ≤ ub2) && return sum_inter - (ub1 - lb2)
   (lb2 ≤ lb1 ≤ ub2 ≤ ub1) && return sum_inter - (ub2 - lb1) # symmetry
    
   return sum_inter
end

add_interv_len (generic function with 1 method)

In [10]:
interv_1 = IntervInt64(1, 10)
interv_2 = IntervInt64(3, 7)
@test add_interv_len(interv_1, interv_2) == 9

interv_1 = IntervInt64(3, 7)
interv_2 = IntervInt64(1, 8)
@test add_interv_len(interv_1, interv_2) == 7

interv_1 = IntervInt64(1, 6)
interv_2 = IntervInt64(3, 9)
@test add_interv_len(interv_1, interv_2) == 8


interv_1 = IntervInt64(6, 10)
interv_2 = IntervInt64(5, 8)
@test add_interv_len(interv_1, interv_2) == 5

interv_1 = IntervInt64(1, 5)
interv_2 = IntervInt64(6, 8)
@test add_interv_len(interv_1, interv_2) == 6

Test Passed

#### Adding interval (length) and a length

In [11]:
function add_intervlen_len(interv::Interval{T}, len::T)::T where T <: Integer
   return interv_length(interv) + len
end

function add_intervlen_len(len::T, interv::Interval{T})::T where T <: Integer
   return interv_length(interv) + len
end

add_intervlen_len (generic function with 2 methods)

In [12]:
interv_1 = IntervInt64(1, 10)
@test add_intervlen_len(interv_1, 10) == 19

interv_1 = IntervInt64(2, 10)
@test add_intervlen_len(6, interv_1) == 14

Test Passed

#### Sorting an array of Intervals

We will sort on ascending order given the lower bound. 

In [13]:
# Sorting
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sort!(ary, by = interv -> interv.lb)

7-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 5)
 Interval{Int64}(1, 3)
 Interval{Int64}(2, 4)
 Interval{Int64}(3, 7)
 Interval{Int64}(4, 8)
 Interval{Int64}(5, 11)
 Interval{Int64}(8, 10)

#### Merging overlapping intervals (in linear time)

In [14]:
function merge_interv(interv_lst::Array{Interval{T}, 1})::Array{Interval{T}, 1} where T <: Integer
  """
  merge all overlapping intervals from given list(array)     
  """
  size(interv_lst, 1) == 1 && return interv_lst
    
  i1, ix = interv_lst[1], 2
  n_interv_lst = []
  
  # complexity linear in length of array
  n = size(interv_lst, 1)
  while ix <= n
    i2 = interv_lst[ix]
    if interv_inter(i1, i2)
      i1 = IntervInt64(min(i1.lb, i2.lb), max(i1.ub, i2.ub))
    else   
      push!(n_interv_lst, i1)
      i1 = i2
    end
    ix += 1
  end
  push!(n_interv_lst, i1)  
  return n_interv_lst
end

merge_interv (generic function with 1 method)

In [15]:
ary1 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sort!(ary1, by = interv -> interv.lb)

ary2 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(17, 20), IntervInt64(11, 15), IntervInt64(3, 7),
]

sort!(ary2, by = interv -> interv.lb)

6-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 5)
 Interval{Int64}(2, 4)
 Interval{Int64}(3, 7)
 Interval{Int64}(8, 10)
 Interval{Int64}(11, 15)
 Interval{Int64}(17, 20)

In [16]:
@test merge_interv(ary1) == [IntervInt64(1, 11)]

Test Passed

In [17]:
@test merge_interv(ary2) == [IntervInt64(1, 7), IntervInt64(8, 10), 
    IntervInt64(11, 15), IntervInt64(17, 20)]

Test Passed

In [18]:
ary3 = [IntervInt64(1, 2), IntervInt64(11, 15)]
sort!(ary3, by = interv -> interv.lb)
@test merge_interv(ary3) == ary3 # Identity

Test Passed

In [19]:
@btime merge_interv(ary1)

  1.340 μs (22 allocations: 832 bytes)


1-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 11)

In [20]:
@btime merge_interv(ary2)

  596.153 ns (13 allocations: 592 bytes)


4-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 7)
 Interval{Int64}(8, 10)
 Interval{Int64}(11, 15)
 Interval{Int64}(17, 20)

#### Merging overlapping intervals (in linear time) - ALT

In [21]:
const VT{T <: Integer} = Vector{Interval{T}}

function merge_interv_alt(il::VT{T})::VT{T} where T <: Integer
	"""
	Merge all the intervals from given vector
	"""
	
	size(il, 1) == 1 && return il   ## identity
	
	n = size(il, 1)
	n_il = VT{T}(undef, n)  ## at most the size of given vector
	i₁, jx = il[1], 1
	
	for ix ∈ 2:n
		i₂ = il[ix]
		if interv_inter(i₁, i₂)
			i₁ = Interval{T}(min(i₁.lb, i₂.lb), max(i₁.ub, i₂.ub))
		else
			n_il[jx] = i₁
			jx += 1
			i₁ = i₂
		end
	end
	
	n_il[jx] = i₁
	view(n_il, 1:jx)
end

merge_interv_alt (generic function with 1 method)

In [22]:
ary1 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sort!(ary1, by = interv -> interv.lb)

ary2 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(17, 20), IntervInt64(11, 15), IntervInt64(3, 7),
]

sort!(ary2, by = interv -> interv.lb)

6-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 5)
 Interval{Int64}(2, 4)
 Interval{Int64}(3, 7)
 Interval{Int64}(8, 10)
 Interval{Int64}(11, 15)
 Interval{Int64}(17, 20)

In [23]:
@test merge_interv_alt(ary1) == [IntervInt64(1, 11)]

Test Passed

In [24]:
@test merge_interv(ary2) == [IntervInt64(1, 7), IntervInt64(8, 10), 
    IntervInt64(11, 15), IntervInt64(17, 20)]

Test Passed

In [25]:
ary3 = [IntervInt64(1, 2), IntervInt64(11, 15)]
sort!(ary3, by = interv -> interv.lb)
@test merge_interv(ary3) == ary3 # Identity

Test Passed

In [26]:
@btime merge_interv_alt(ary1)

  82.979 ns (2 allocations: 288 bytes)


1-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 11)

In [27]:
@btime merge_interv_alt(ary2)

  85.869 ns (2 allocations: 320 bytes)


4-element Array{Interval{Int64},1}:
 Interval{Int64}(1, 7)
 Interval{Int64}(8, 10)
 Interval{Int64}(11, 15)
 Interval{Int64}(17, 20)

This version is more efficient in term of space and speed (expected with Julia).

#### Extend +

In [28]:
import Base: +, convert #promote_rule

# overload: 
+(i1::Interval{T}, i2::Interval{T})  where T <: Integer = add_interv_len(i1, i2)

+(len::T, iv::Interval{T})  where T <: Integer = add_intervlen_len(len, iv)
+(iv::Interval{T}, len::T)  where T <: Integer = add_intervlen_len(iv, len)

convert(::Type{T}, i1::Interval{T}) where T <: Integer = i1.ub - i1.lb
# promote_rule(::Interval{T}, ::Interval{<:Integer}) = T

convert (generic function with 184 methods)

In [29]:
@test IntervInt64(2, 4) + IntervInt64(8, 10) == 4

@test IntervInt64(2, 4) + 3 == 5
@test 3 + IntervInt64(2, 4) == 5

Test Passed

#### Compute Sum of Interval array - 1

In [30]:
function sum_intervals_1(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv)::T where T <: Integer
    size(interv_lst, 1) == 1 && return interv_length(interv_lst[1])
    
    sort!(interv_lst, by = interv -> interv.lb)
    n_interv_lst = merger_fun(interv_lst)
    s = interv_length(n_interv_lst[1])
    for iv in n_interv_lst[2:end]
        s += interv_length(iv)
    end
    return s
end

sum_intervals_1 (generic function with 2 methods)

In [31]:
function sum_intervals_1_alt(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv_alt)::T where T <: Integer
  return sum_intervals_1(interv_lst, merger_fun)
end

sum_intervals_1_alt (generic function with 2 methods)

In [32]:
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

c_ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

@test sum_intervals_1(ary) == sum_intervals_1_alt(c_ary) # um_intervals_1_alt( side-effects c_ary

Test Passed

In [33]:
ary1 = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_1(ary1) == 5

ary2 = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_1(ary2) == 9

ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_1(ary3) == 19

Test Passed

In [34]:
ary1 = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_1_alt(ary1) == 5

ary2 = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_1_alt(ary2) == 9

ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_1_alt(ary3) == 19

Test Passed

In [35]:
ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]

@btime sum_intervals_1(ary3)

  1.001 μs (18 allocations: 800 bytes)


19

In [36]:
ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]

@btime sum_intervals_1_alt(ary3)

  145.813 ns (4 allocations: 416 bytes)


19

#### Compute Sum of Interval array - 2

In [37]:
function sum_intervals_2(interv_lst::Array{Interval{T}, 1}, merger_fn=merge_interv)::T where T <: Integer
  size(interv_lst, 1) == 1 && return interv_length(interv_lst[1])
    
  n_interv_lst = merger_fn(sort(interv_lst, by = interv -> interv.lb))
  sum(map(iv -> interv_length(iv), 
      n_interv_lst))
end

sum_intervals_2 (generic function with 2 methods)

In [38]:
function sum_intervals_2_alt(interv_lst::Array{Interval{T}, 1}, 
        merger_fun=merge_interv_alt)::T where T <: Integer
  return sum_intervals_2(interv_lst, merger_fun)
end

sum_intervals_2_alt (generic function with 2 methods)

In [39]:
ary1 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

# real copy
cary1 =  [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sum_intervals_2(ary1) == sum_intervals_2_alt(cary1)  # sum_intervals_2_alt side-effects input array!

true

In [40]:
ary1 = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_2(ary1) == 5

ary2 = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_2(ary2) == 9

ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_2(ary3) == 19

Test Passed

In [41]:
ary1 = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_2_alt(ary1) == 5

ary2 = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_2_alt(ary2) == 9

ary3 = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_2_alt(ary3) == 19

Test Passed

In [42]:
ary1 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

@btime sum_intervals_2_alt(ary1)

  185.952 ns (5 allocations: 656 bytes)


10

In [43]:
ary1 = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

@btime sum_intervals_2(ary1)

  1.383 μs (25 allocations: 1.17 KiB)


10

#### Compute Sum of Interval array - 3

In [44]:
function sum_intervals_3(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv)::T where T <: Integer
  size(interv_lst, 1) == 1 && return interv_length(interv_lst[1])
    
  sort!(interv_lst, by = interv -> interv.lb)
  n_interv_lst = merger_fun(interv_lst)
    
  return reduce(+, n_interv_lst)
end

sum_intervals_3 (generic function with 2 methods)

In [45]:
function sum_intervals_3_alt(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv_alt)::T where T <: Integer
    return sum_intervals_3(interv_lst, merger_fun)
end

sum_intervals_3_alt (generic function with 2 methods)

In [46]:
ary = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_3(ary) == 5

ary = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_3(ary) == 9

ary = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_3(ary) == 19

Test Passed

In [47]:
ary = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_3_alt(ary) == 5

ary = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_3_alt(ary) == 9

ary = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_3_alt(ary) == 19

Test Passed

In [48]:
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sum_intervals_3(ary)

10

In [49]:
@btime sum_intervals_3(ary)

  1.398 μs (23 allocations: 912 bytes)


10

In [50]:
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

@btime sum_intervals_3_alt(ary)

  129.613 ns (3 allocations: 368 bytes)


10

#### Compute Sum of Interval array - 4

In [51]:
function sum_intervals_4(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv)::T where T <: Integer
  size(interv_lst, 1) == 1 && return interv_length(interv_lst[1])
    
  sort!(interv_lst, by = interv -> interv.lb)
  n_interv_lst = merger_fun(interv_lst)
    
  return sum(interv_length.(n_interv_lst))
end

sum_intervals_4 (generic function with 2 methods)

In [52]:
function sum_intervals_4_alt(interv_lst::Array{Interval{T}, 1},
        merger_fun=merge_interv_alt)::T where T <: Integer
    return sum_intervals_4(interv_lst, merger_fun)
end

sum_intervals_4_alt (generic function with 2 methods)

In [53]:
ary = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_4(ary) == 5

ary = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_4(ary) == 9

ary = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_4(ary) == 19

Test Passed

In [54]:
ary = [IntervInt64(1, 2), IntervInt64(11, 15)]
@test sum_intervals_4_alt(ary) == 5

ary = [IntervInt64(1, 2), IntervInt64(11, 15), IntervInt64(6, 10)]
@test sum_intervals_4_alt(ary) == 9

ary = [
    IntervInt64(1, 5), IntervInt64(10, 20), IntervInt64(1, 6),
    IntervInt64(16, 19), IntervInt64(5, 11)
]
@test sum_intervals_4_alt(ary) == 19

Test Passed

In [55]:
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

sum_intervals_4(ary)

10

In [56]:
@btime sum_intervals_4(ary)

  1.410 μs (24 allocations: 1008 bytes)


10

In [57]:
ary = [
    IntervInt64(2, 4), IntervInt64(8, 10), IntervInt64(1, 5),
    IntervInt64(1, 3), IntervInt64(4, 8), IntervInt64(3, 7),
    IntervInt64(5, 11)
]

@btime sum_intervals_4_alt(ary)

  153.737 ns (4 allocations: 464 bytes)


10